<a href="https://colab.research.google.com/github/sagar9926/EPAI_session16/blob/main/EPAI_session_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 1 :

The files are:

- personal_info.csv - personal information such as name, gender, etc. (one row per person)

- vehicles.csv - what vehicle people own (one row per person)

- employment.csv - where a person is employed (one row per person)

- update_status.csv - when the person's data was created and last updated

Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

In [1]:
!git clone https://github.com/sagar9926/EPAI_session16.git

Cloning into 'EPAI_session16'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [2]:
cd EPAI_session16

/content/EPAI_session16


In [3]:
!ls

employment.csv	       personal_info.csv  update_status.csv
EPAI_session_16.ipynb  README.md	  vehicles.csv


In [4]:
from collections import namedtuple
import datetime
import csv

In [5]:
def read_file(file_name):
  with open(file_name) as f:
    rows = csv.reader(f,delimiter = ',',quotechar = '"')
    yield from rows


In [6]:
data_type_dict = {
    "employment.csv" : [str,str,str,str],
    'personal_info.csv' : [str,str,str,str,str],
    'update_status.csv' : [str,datetime.datetime,datetime.datetime],
    'vehicles.csv' : [str,	str,	str,	str]

}
date_format = '%Y-%m-%dT%H:%M:%SZ'

In [7]:
class file_iter:
  def __init__(self,file_name):
    self.file_name = file_name
    self.data_type = data_type_dict[self.file_name]
    self.file = read_file(self.file_name)
    self.columns = " ".join(next(self.file))
    self.row_name_tuple = namedtuple((self.file_name[0].upper() + self.file_name[1:-4]),self.columns)
    self.row_index = 0
  
  def __iter__(self):
    for row in self.file:
      if self.file_name != 'update_status.csv':
        row = [type_(data) for type_ , data in zip(self.data_type,row)]
      else :
        row = [type_(data) if type_ == str else type_(data,date_format) for type_ , data in zip(self.data_type,row)]
      yield self.row_name_tuple(*row)



### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

In [8]:
employment_iter = file_iter('employment.csv')
personal_info_iter = file_iter('personal_info.csv')
update_status_iter = file_iter('update_status.csv')
vehicles_iter = file_iter('vehicles.csv')

In [12]:
for i in update_status_iter:
  print(i)

TypeError: ignored

In [21]:
datetime.datetime.strptime('2017-10-07',format = "%Y-%m-%d")

TypeError: ignored

In [24]:
from datetime import datetime

datetime.strptime('2017-10-07T00:14:42Z', '%Y-%m-%dT%H:%M:%SZ')

datetime.datetime(2017, 10, 7, 0, 14, 42)

In [ ]:
date_format = '%Y/%m/%d %H:%M:%S
